In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, confusion_matrix
from sklearn.preprocessing import label_binarize

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [2]:
from art.defences.preprocessor.gaussian_augmentation import GaussianAugmentation

gaussian_augmenter = GaussianAugmentation()

def preprocess(X,y, gaussian_augmenter):
    x_test, y_test = gaussian_augmenter(X, y)

    return torch.tensor(x_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.long)


In [3]:
head = {
            "model" : '',
            "attack_model": '',
            'epsilon': '',
            'Accuracy': '',
            'Macro Precision': '',
            'Weighted Precision': '',
            'Macro Recall': '',
            'Weighted Recall': '',
            'Macro F1': '',
            'Weighted F1': '',

        }
head = pd.DataFrame([head])
head.to_csv("./gaussian_augmenter.csv", mode='a', index=False)




In [4]:
def calculate_performance_metrics(X_test, y_test, model, model_name, attack_name, eps):
    model.eval()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    all_preds = []
    all_labels = []
    probabilities = []

    num_classes = len(np.unique(y_test))
    
    # X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
    X_test_tensor , y_test_tensor= preprocess(X_test, y_test, gaussian_augmenter)
    X_test_tensor = X_test_tensor[:len(X_test)] 
    y_test_tensor = y_test_tensor[:len(y_test)] 
    X_test_tensor = X_test_tensor.to(device)
    y_test_tensor = y_test_tensor.to(device)

    # X_test_tensor = preprocess(X_test, gaussian_augmenter).to(device)
    # y_test_tensor = torch.tensor(y_test, dtype=torch.long).to(device)
    
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
    test_loader = DataLoader(dataset=test_dataset)

    with torch.no_grad():
        
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            probabilities.extend(torch.nn.functional.softmax(outputs, dim=1).cpu().numpy())
        
        all_preds = np.array(all_preds)
        all_labels = np.array(all_labels)
        probabilities = np.array(probabilities)
        
        accuracy = accuracy_score(all_labels, all_preds)

        precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(all_labels, all_preds, average='macro')
        precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')
    
        

        print(f"Accuracy: {accuracy}")
        
        print("\nmacro")
        print(f"Precision: {precision_macro}\nRecall: {recall_macro}\nF1 Score: {f1_macro}")
    
        print("\nweighted")
        print(f"Precision: {precision_weighted}\nRecall: {recall_weighted}\nF1 Score: {f1_weighted}")
        print()
        


        new_row = {
            "model" : model_name,
            "attack_model" : attack_name,
            'epsilon': eps,
            'Accuracy': accuracy,
            'Macro Precision': precision_macro,
            'Weighted Precision': precision_weighted,
            'Macro Recall': recall_macro,
            'Weighted Recall': recall_weighted,
            'Macro F1': f1_macro,
            'Weighted F1': f1_weighted,

        }
        new_row_df = pd.DataFrame([new_row])
        new_row_df.to_csv("./gaussian_augmenter.csv", mode='a', index=False, header=False)




In [5]:
# def calculate_performance_metrics(X_test, y_test, model, model_name, attack_name, eps):
#     model.eval()
#     device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#     model.to(device)
    
#     all_preds = []
#     all_labels = []
#     probabilities = []

#     num_classes = len(np.unique(y_test))
    
#     # X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
#     X_test_tensor , y_test_tensor= preprocess(X_test, y_test, gaussian_augmenter)
#     X_test_tensor = X_test_tensor[:len(X_test)] 
#     y_test_tensor = y_test_tensor[:len(y_test)] 
#     X_test_tensor = X_test_tensor.to(device)
#     y_test_tensor = y_test_tensor.to(device)

#     # X_test_tensor = preprocess(X_test, gaussian_augmenter).to(device)
#     # y_test_tensor = torch.tensor(y_test, dtype=torch.long).to(device)
    
#     test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
#     test_loader = DataLoader(dataset=test_dataset)

#     with torch.no_grad():
        
#         for inputs, labels in test_loader:
#             inputs, labels = inputs.to(device), labels.to(device)
#             outputs = model(inputs)
#             preds = torch.argmax(outputs, dim=1)
#             all_preds.extend(preds.cpu().numpy())
#             all_labels.extend(labels.cpu().numpy())
#             probabilities.extend(torch.nn.functional.softmax(outputs, dim=1).cpu().numpy())
        
#         all_preds = np.array(all_preds)
#         all_labels = np.array(all_labels)
#         probabilities = np.array(probabilities)

#         np.save(f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Defense_Label/UNSW_Def6/y_pred_{attack_name}{eps}_Def6.npy", all_preds)


In [6]:
x_test = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
x_train = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_train.npy')
x_val = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_val.npy')
y_test = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')
y_train = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_train.npy')
y_val = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_val.npy')

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

Using cuda device


In [8]:
input_shape = x_train.shape[1]
output_shape = len(np.unique(y_train))

In [9]:
# x_train_tensor = torch.tensor(x_train, dtype=torch.float32).to(device)
x_train_tensor , y_train_tensor= preprocess(x_train, y_train, gaussian_augmenter)
# y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)
x_train_tensor = x_train_tensor.to(device)
y_train_tensor = y_train_tensor.to(device)

# x_val_tensor = torch.tensor(x_val, dtype=torch.float32).to(device)
x_val_tensor , y_val_tensor= preprocess(x_val,y_val, gaussian_augmenter)
# y_val_tensor = torch.tensor(y_val, dtype=torch.long).to(device)
x_val_tensor = x_val_tensor.to(device)
y_val_tensor = y_val_tensor.to(device)


train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True)

val_dataset = TensorDataset(x_val_tensor, y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=100, shuffle=True)

In [10]:
class DNNModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(DNNModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, 30)
        self.fc3 = nn.Linear(30, 20)
        self.fc4 = nn.Linear(20, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

model = DNNModel(input_size=input_shape, output_size=output_shape).to(device)

# Compile model
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_function = nn.CrossEntropyLoss()

# Early stopping variables
min_delta = 0.001
patience = 5
patience_counter = 0
best_loss = float('inf')

In [11]:
# for epoch in range(50):
#     model.train()
#     train_loss = 0.0
#     for inputs, labels in train_loader:
#         inputs, labels = inputs.to(device), labels.to(device)
#         optimizer.zero_grad()
#         outputs = model(inputs)
#         loss = loss_function(outputs, labels)
#         loss.backward()
#         optimizer.step()
#         train_loss += loss.item()

#     avg_train_loss = train_loss / len(train_loader)

#     model.eval()
#     val_train_loss = 0.0
#     correct_predictions = 0
#     with torch.no_grad():
#         for inputs, labels in val_loader:
#             inputs, labels = inputs.to(device), labels.to(device)
#             outputs = model(inputs)
#             loss = loss_function(outputs, labels)
#             val_train_loss += loss.item()
#             _, predicted = torch.max(outputs.data, 1)
#             correct_predictions += (predicted == labels).sum().item()

#     avg_val_loss = val_train_loss / len(val_loader)
#     val_accuracy = correct_predictions / len(val_dataset)

#     print(f"Epoch {epoch+1}, Training Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

#     # Early stopping check using min_delta
#     if best_loss - avg_val_loss > min_delta:
#         best_loss = avg_val_loss
#         patience_counter = 0
#     else:
#         patience_counter += 1

#     if patience_counter >= patience:
#         print("Early stopping triggered")
#         break

In [12]:

model.load_state_dict(torch.load("/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Defense/preprocessor/dnn_pytorch_gaussian_augmenter.pt"))

/tmp/ipykernel_174/3661704067.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Defense/pre

<All keys matched successfully>

In [13]:
# calculate_performance_metrics(x_test, y_test, model, 'DNN', 'Baseline', '0')

In [14]:
def print_empty_file():
    new_row = {
        "model" : "0",
        "attack_model" : "0",
        'epsilon': "0",
        'Accuracy': "0",
        'Macro Precision': "0",
        'Weighted Precision': "0",
        'Macro Recall': "0",
        'Weighted Recall': "0",
        'Macro F1': "0",
        'Weighted F1': "0",
    }
    new_row_df = pd.DataFrame([new_row])
    new_row_df.to_csv("./gaussian_augmenter.csv", mode='a', index=False, header=False)

In [13]:
epsilon = 0
Def = "Def6"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

percentage = ["100", "50", "20", "1"]
model_name = ["XGB", "RF", "DT" ]

base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

for m_name in model_name:
    for p in percentage:
        for attack in attack_names:
            print(f"start {m_name} {p} {attack}")
            
            x_path = f"{base_path}/{m_name}/UNSW_Input{p}/x_test_adv_{attack}_{Def}.npy"
            y_path = f"{base_path}/{m_name}/UNSW_Input{p}/y_test_adv_{attack}_{Def}.npy"

            try:
                x_test_adv = np.load(x_path)
                y_test_adv = np.load(y_path)
                print(x_test_adv.shape, y_test_adv.shape)

                m_per_name = f"{m_name}{p}"
                calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
            except FileNotFoundError:
                print(x_path, "not found")
                print_empty_file()


start XGB 100 baseline
(2019, 56) (2019,)
Accuracy: 0.8350668647845468

macro
Precision: 0.8080879466038476
Recall: 0.8713235650983779
F1 Score: 0.8189728637138234

weighted
Precision: 0.8823009832192911
Recall: 0.8350668647845468
F1 Score: 0.8425791808989543

start XGB 100 BIM
(175150, 56) (175150,)
Accuracy: 0.8252355124179275

macro
Precision: 0.8233471521916685
Recall: 0.8561139746558775
F1 Score: 0.8207502703610504

weighted
Precision: 0.8681721030624897
Recall: 0.8252355124179275
F1 Score: 0.8293581275273056

start XGB 100 FGSM
(189248, 56) (189248,)
Accuracy: 0.8668413933040243

macro
Precision: 0.8558508794374879
Recall: 0.8774719590512814
F1 Score: 0.8616061278908633

weighted
Precision: 0.8799126863749956
Recall: 0.8668413933040243
F1 Score: 0.8687936659909687

start XGB 100 PGD
(175150, 56) (175150,)
Accuracy: 0.8252355124179275

macro
Precision: 0.8233471521916685
Recall: 0.8561139746558775
F1 Score: 0.8207502703610504

weighted
Precision: 0.8681721030624897
Recall: 0.82523

In [12]:
epsilon = 0
Def = "Def6"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

percentage = ["100", "50", "20", "1"]
model_name = ["XGB", "RF", "DT" ]

base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

for m_name in model_name:
    for p in percentage:
        for attack in attack_names:
            print(f"start {m_name} {p} {attack}")
            
            x_path = f"{base_path}/{m_name}_ExcludeCaFA/UNSW_Input{p}/x_test_adv_{attack}_{Def}.npy"
            y_path = f"{base_path}/{m_name}_ExcludeCaFA/UNSW_Input{p}/y_test_adv_{attack}_{Def}.npy"

            try:
                x_test_adv = np.load(x_path)
                y_test_adv = np.load(y_path)
                print(x_test_adv.shape, y_test_adv.shape)

                m_per_name = f"{m_name}{p}_ExcludeCaFA"
                calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
            except FileNotFoundError:
                print(x_path, "not found")
                print_empty_file()


start XGB 100 baseline
(1107, 56) (1107,)
Accuracy: 0.6233062330623306

macro
Precision: 0.564691943127962
Recall: 0.5123344802418934
F1 Score: 0.4359528595255405

weighted
Precision: 0.5806457827611451
Recall: 0.6233062330623306
F1 Score: 0.516121309330468

start XGB 100 BIM
(152238, 56) (152238,)
Accuracy: 0.8250502502660308

macro
Precision: 0.7967399208690857
Recall: 0.8645113185056978
F1 Score: 0.8070522266817864

weighted
Precision: 0.8785908665278238
Recall: 0.8250502502660308
F1 Score: 0.8337976551386382

start XGB 100 FGSM
(166606, 56) (166606,)
Accuracy: 0.8887314982653686

macro
Precision: 0.8757373604844414
Recall: 0.8981741784551147
F1 Score: 0.8830264350826871

weighted
Precision: 0.898916810167495
Recall: 0.8887314982653686
F1 Score: 0.8903378594811261

start XGB 100 PGD
(152238, 56) (152238,)
Accuracy: 0.8250502502660308

macro
Precision: 0.7967399208690857
Recall: 0.8645113185056978
F1 Score: 0.8070522266817864

weighted
Precision: 0.8785908665278238
Recall: 0.82505025

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.10180995475113122

macro
Precision: 0.5
Recall: 0.05090497737556561
F1 Score: 0.09240246406570841

weighted
Precision: 1.0
Recall: 0.10180995475113122
F1 Score: 0.18480492813141683

start XGB 100 SINIFGSM
(148082, 56) (148082,)
Accuracy: 0.7841668805121487

macro
Precision: 0.7120470571709827
Recall: 0.8336498238555675
F1 Score: 0.7268035733922009

weighted
Precision: 0.880841456879825
Recall: 0.7841668805121487
F1 Score: 0.8071857514806705

start XGB 100 VNIFGSM
(170599, 56) (170599,)
Accuracy: 0.8600577963528508

macro
Precision: 0.8450574326176248
Recall: 0.8815414032562583
F1 Score: 0.852520594765851

weighted
Precision: 0.8856972735639534
Recall: 0.8600577963528508
F1 Score: 0.863531002632939

start XGB 50 baseline
(1083, 56) (1083,)
Accuracy: 0.6140350877192983

macro
Precision: 0.5592042186001918
Recall: 0.5088858509911142
F1 Score: 0.421302782692288

weighted
Precision: 0.5727069351230425
Recall: 0.6140350877192983
F1 Score: 0.49747058963896246

start XGB 50 BIM
(16

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.06751054852320675

macro
Precision: 0.5
Recall: 0.03375527426160337
F1 Score: 0.06324110671936758

weighted
Precision: 1.0
Recall: 0.06751054852320675
F1 Score: 0.12648221343873517

start RF 1 SINIFGSM
(183365, 56) (183365,)
Accuracy: 0.7872331142802607

macro
Precision: 0.7644749099148527
Recall: 0.8463889243860434
F1 Score: 0.7663366182989031

weighted
Precision: 0.8732663957805255
Recall: 0.7872331142802607
F1 Score: 0.8008993883549109

start RF 1 VNIFGSM
(198755, 56) (198755,)
Accuracy: 0.8032401700586149

macro
Precision: 0.7903786080208792
Recall: 0.8463607516139242
F1 Score: 0.7913544428861694

weighted
Precision: 0.8649590782219261
Recall: 0.8032401700586149
F1 Score: 0.8116294318933254

start DT 100 baseline
(1525, 56) (1525,)
Accuracy: 0.8511475409836066

macro
Precision: 0.9242828552368245
Recall: 0.5513833992094862
F1 Score: 0.5522300218985938

weighted
Precision: 0.8736889073590044
Recall: 0.8511475409836066
F1 Score: 0.7966898549223722

start DT 100 BIM
(12902

In [12]:
epsilon = 0
Def = "Def6"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

percentage = ["100", "50", "20", "1"]
model_name = ["XGB", "RF", "DT" ]

base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"


total_num_attack = ["2", "3"]
for num_attack in total_num_attack:
    for m_name in model_name:
        for p in percentage:
            for attack in attack_names:
                print(f"start {m_name} {p} {attack}")
                
                x_path = f"{base_path}/{m_name}_Exclude{num_attack}Attack/UNSW_Input{p}/x_test_adv_{attack}_{Def}.npy"
                y_path = f"{base_path}/{m_name}_Exclude{num_attack}Attack/UNSW_Input{p}/y_test_adv_{attack}_{Def}.npy"
    
                try:
                    x_test_adv = np.load(x_path)
                    y_test_adv = np.load(y_path)
                    print(x_test_adv.shape, y_test_adv.shape)
    
                    m_per_name = f"{m_name}{p}_Exclude{num_attack}Attack"
                    calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
                except FileNotFoundError:
                    print(x_path, "not found")
                    print_empty_file()


start XGB 100 baseline
(178, 56) (178,)
Accuracy: 0.8314606741573034

macro
Precision: 0.9147727272727273
Recall: 0.53125
F1 Score: 0.5122396784800877

weighted
Precision: 0.8601889683350358
Recall: 0.8314606741573034
F1 Score: 0.7649563000578833

start XGB 100 BIM
(148980, 56) (148980,)
Accuracy: 0.8098670962545308

macro
Precision: 0.8105721904676819
Recall: 0.8476581661803105
F1 Score: 0.8047961487907831

weighted
Precision: 0.8643516490889405
Recall: 0.8098670962545308
F1 Score: 0.8150727790373636

start XGB 100 FGSM
(175201, 56) (175201,)
Accuracy: 0.8794242041997478

macro
Precision: 0.8641159118052872
Recall: 0.8923946058715222
F1 Score: 0.8724722726559472

weighted
Precision: 0.8939799194063398
Recall: 0.8794242041997478
F1 Score: 0.8816643012436736

start XGB 100 PGD
(148980, 56) (148980,)
Accuracy: 0.8098670962545308

macro
Precision: 0.8105721904676819
Recall: 0.8476581661803105
F1 Score: 0.8047961487907831

weighted
Precision: 0.8643516490889405
Recall: 0.8098670962545308
F

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.8306241895349245

macro
Precision: 0.8149631113869407
Recall: 0.8646216258944471
F1 Score: 0.8203337235386992

weighted
Precision: 0.8754561585977279
Recall: 0.8306241895349245
F1 Score: 0.8366821916590532

start XGB 1 FGSM
(179548, 56) (179548,)
Accuracy: 0.8854735224006951

macro
Precision: 0.8751980115167703
Recall: 0.8923062052065431
F1 Score: 0.8808559210065741

weighted
Precision: 0.8932915143963454
Recall: 0.8854735224006951
F1 Score: 0.886746050626008

start XGB 1 PGD
(142665, 56) (142665,)
Accuracy: 0.8306241895349245

macro
Precision: 0.8149631113869407
Recall: 0.8646216258944471
F1 Score: 0.8203337235386992

weighted
Precision: 0.8754561585977279
Recall: 0.8306241895349245
F1 Score: 0.8366821916590532

start XGB 1 DF
(247595, 56) (247595,)
Accuracy: 0.5840788384256548

macro
Precision: 0.5658611204304193
Recall: 0.5666246103416215
F1 Score: 0.5661317575140246

weighted
Precision: 0.5866854122271222
Recall: 0.5840788384256548
F1 Score: 0.5852770449515579

start XG

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.8270110432735555

macro
Precision: 0.785765635526217
Recall: 0.8534402420458663
F1 Score: 0.801228756250401

weighted
Precision: 0.8711845215831285
Recall: 0.8270110432735555
F1 Score: 0.83600247457896

start XGB 1 VNIFGSM
(141793, 56) (141793,)
Accuracy: 0.8796273440860973

macro
Precision: 0.8763267293930501
Recall: 0.888681588953888
F1 Score: 0.8780517865343314

weighted
Precision: 0.8901226941658046
Recall: 0.8796273440860973
F1 Score: 0.8805945567446053

start RF 100 baseline
(1207, 56) (1207,)
Accuracy: 0.9047224523612262

macro
Precision: 0.9106345885634588
Recall: 0.5435378927911275
F1 Score: 0.5550394455503944

weighted
Precision: 0.9058519052620755
Recall: 0.9047224523612262
F1 Score: 0.8677933123896149

start RF 100 BIM
(175109, 56) (175109,)
Accuracy: 0.8151265783026572

macro
Precision: 0.7953305952754881
Recall: 0.8659226373321087
F1 Score: 0.7999670081277002

weighted
Precision: 0.8824752922290121
Recall: 0.8151265783026572
F1 Score: 0.824673717474991

start 

...

Accuracy: 0.8161512027491409

macro
Precision: 0.5
Recall: 0.40807560137457044
F1 Score: 0.44938505203405865

weighted
Precision: 1.0
Recall: 0.8161512027491409
F1 Score: 0.8987701040681173

start RF 1 ZOO
(39, 56) (39,)
Accuracy: 0.8461538461538461

macro
Precision: 0.4583333333333333
Recall: 0.4583333333333333
F1 Score: 0.4583333333333333

weighted
Precision: 0.8461538461538461
Recall: 0.8461538461538461
F1 Score: 0.8461538461538461

start RF 1 CaFA
(1, 56) (1,)
Accuracy: 0.0

macro
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

weighted
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

start RF 1 SINIFGSM
(4177, 56) (4177,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.985156811108451

macro
Precision: 0.5
Recall: 0.4925784055542255
F1 Score: 0.49626145682585626

weighted
Precision: 1.0
Recall: 0.985156811108451
F1 Score: 0.9925229136517124

start RF 1 VNIFGSM
(4394, 56) (4394,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9415111515703232

macro
Precision: 0.5
Recall: 0.4707555757851616
F1 Score: 0.4849372875395616

weighted
Precision: 1.0
Recall: 0.9415111515703232
F1 Score: 0.9698745750791232

start DT 100 baseline
(1306, 56) (1306,)
Accuracy: 0.8453292496171516

macro
Precision: 0.890843023255814
Recall: 0.534263506625892
F1 Score: 0.5222181976238771

weighted
Precision: 0.8596192884361978
Recall: 0.8453292496171516
F1 Score: 0.7851595937524823

start DT 100 BIM
(6716, 56) (6716,)
Accuracy: 0.8916021441334128

macro
Precision: 0.5928810887264339
Recall: 0.7969684993237514
F1 Score: 0.624590133724823

weighted
Precision: 0.9605468225377017
Recall: 0.8916021441334128
F1 Score: 0.91922732741368

start DT 100 FGSM
(4768, 56) (4768,)
Accuracy: 0.7019714765100671

macro
Precision: 0.5260512433015986
Recall: 0.6573404090292686
F1 Score: 0.47500863772382823

weighted
Precision: 0.9463579626429125
Recall: 0.7019714765100671
F1 Score: 0.7950009016584935

start DT 100 PGD
(6716, 56) (6716,)
Accuracy

In [15]:
epsilon = 0

Def = "Def6"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

rec_list = ["Euclidean", "cosine", "manhattan"]
for rec in rec_list:
    for attack in attack_names:
        print(f"start {attack}")
        
        x_path = f"{base_path}/Recommendation_{rec}/x_test_adv_{attack}_{Def}.npy"
        y_path = f"{base_path}/Recommendation_{rec}/y_test_adv_{attack}_{Def}.npy"
    
        try:
            x_test_adv = np.load(x_path)
            y_test_adv = np.load(y_path)
            m_per_name = f"Recommendation_{rec}"
            calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
        except FileNotFoundError:
            print(x_path, "not found")
            print_empty_file()



start baseline
Accuracy: 0.9304473304473304

macro
Precision: 0.9332039911308204
Recall: 0.93664563617245
F1 Score: 0.930391649448624

weighted
Precision: 0.939739011911961
Recall: 0.9304473304473304
F1 Score: 0.9305842602502721

start BIM
Accuracy: 0.9059517216666124

macro
Precision: 0.8254578199167844
Recall: 0.8386613153122409
F1 Score: 0.8318240434946158

weighted
Precision: 0.9080077702428391
Recall: 0.9059517216666124
F1 Score: 0.9068865015140217

start FGSM
Accuracy: 0.8354272889170794

macro
Precision: 0.8340650322929191
Recall: 0.8377258660883562
F1 Score: 0.834621655766569

weighted
Precision: 0.8388356099751674
Recall: 0.8354272889170794
F1 Score: 0.8358750193292819

start PGD
Accuracy: 0.9059517216666124

macro
Precision: 0.8254578199167844
Recall: 0.8386613153122409
F1 Score: 0.8318240434946158

weighted
Precision: 0.9080077702428391
Recall: 0.9059517216666124
F1 Score: 0.9068865015140217

start DF
Accuracy: 0.6136676495419026

macro
Precision: 0.5978471028080425
Recall: 

In [14]:

# torch.save(model.state_dict(), "./dnn_pytorch_gaussian_augmenter.pt")

In [16]:
# import time

# epsilon_values = [0.01, 0.1, 0.2, 0.3]

# start_time = time.time()

# for epsilon in epsilon_values:
#     filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_BIM_eps_{epsilon}.npy'
#     x_test_adv = np.load(filename)

#     calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'BIM', epsilon)

# end_time = time.time()
# result = end_time - start_time
# print(f"Execution Time: {result:.6f} seconds")